# Feauture engineering

conda activate tf310
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import tensorflow_hub as hub
import tensorflow as tf

In [7]:
# Load with utf-8 (ignore errors if some lines are corrupted)
df = pd.read_csv("Hair Care Survey_v2 (Responses) - Form responses 1.csv", encoding="utf-8", on_bad_lines="skip")

# Function to clean non-ASCII characters
def clean_text(x):
    if isinstance(x, str):
        return (x.encode("ascii", "ignore")  # drop non-ascii
                 .decode("ascii")
                 .replace("’", "'")
                 .replace("‘", "'")
                 .replace("“", '"')
                 .replace("”", '"')
                 .replace("–", "-")
                 .replace("—", "-"))
    return x

# Apply cleaning to every cell
df = df.applymap(clean_text)

# Save a new clean CSV (safe for MySQL)
df.to_csv("Products_clean.csv", index=False, encoding="utf-8")

print("✅ File cleaned and saved as mysql_clean.csv")


✅ File cleaned and saved as mysql_clean.csv


/var/folders/sd/2tmrf4ds7nqg7lk_wdy856vm0000gn/T/ipykernel_57354/4218053272.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_text)


In [2]:
import pandas as pd

orig_df = pd.read_csv('Hair Care Survey_v2 (Responses) - Form responses 1.csv')


In [9]:
# Read CSV
df = pd.read_csv("HAIRSURVEY_clean.csv")

# Clean column names: strip whitespace, replace spaces with underscores, remove special chars
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_", regex=True)
    .str.replace(r"[^0-9A-Za-z_]", "", regex=True)  # keep original case
)

# Save cleaned CSV (safe for MySQL import)
df.to_csv("HAIRSURVEY_clean2.csv", index=False, encoding="utf-8")


## Step 2: Data Cleaning and nomalization

In [22]:
# Clean column names: strip whitespace, convert to lowercase, replace spaces and special chars

# Drop irrelevant columns (e.g., email, survey codes, etc.)
columns_to_drop = [
    'Timestamp', 'Score', 'Email', 'Email address',
    "SurveyCircle  \n  'Thank-you-for-your-participation' page. Please use the following text line that contains your Survey Code:",
    'Column 53', 'Would you participate in follow-up studies?'
]

Survey_df = orig_df

Survey_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

#Normalize Text Columns: Strip whitespace, Lowercase, Remove special character
Survey_df.columns = (
    Survey_df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('[^a-zA-Z0-9_]', '', regex=True)
)

# Identify text/categorical columns to normalize values (object or category type)
text_cols = Survey_df.select_dtypes(include=['object', 'category']).columns

# Normalize text values in each column
for col in text_cols:
    Survey_df[col] = (
        Survey_df[col]
        .astype(str)
        .str.lower()
        .str.strip('how_often_do_you_')
        .str.replace('none', 'no')     # optional: unify 'none' and 'no'
        .str.replace('NaN', '')        # optional: clear any NaNs as string
    )
#Survey_df.head()

,age,race,gender,country,hair_type,hair_porosity,hair_texture,hair_density,harline_condition,hair_edges_condition,...,family_history_of_hair_loss_or_slow_growth,eating_diet,consumed_waterday_l,satin_scarfbonnet_or_pillowcase,main_factor_influencing_your_hair_health_or_growth,tips_or_products_have_worked_well_for_your_hair,hair_state_and_their_cause_hydrated__healthy,hair_state_and_their_cause_promote_frizzy,hair_state_and_their_cause_tangled,hair_state_and_their_cause_dryness__breaking
0,25-34,black/africa,mal,amibia,4c,l,medium,l,NaN,al,...,,al,NaN,s,a,a,a,a,a,a
1,35-44,black/africa,mal,amibia,3c,medium,medium,medium,2.0,al,...,,modera,1.5,s,moisturising,gentl,oil treatm,shamp,shamp,shamp
2,25-34,black/africa,mal,amibia,3c,medium,medium,medium,2.0,al,...,a,al,1.5,s,moisturizer,a,oil treatm,shamp,shamp,shamp
3,25-34,black/africa,mal,a,3c,ig,coars,ig,1.0,al,...,,modera,2.0,s,lack of consistenc,a,rbal oil,air gel,shamp,shamp
4,25-34,black/africa,mal,amibia,4b,medium,coars,l,2.0,inning,...,,modera,0.5,,lifestyle,castor oil and changing the hair i braid with....,air masks,shamp,leave-in conditioner,air gel


## Step 3: Columns grouping

# renaming columns

In [26]:
# Define columns to encode,

numeric_columns = [ 'consumed_waterday_l']

ordinal_columns = [ 'current_hair_condition', 'age', 'hair_porosity','hair_texture', 'hair_density', 'harline_condition',
                 'hair_breakage', 'hair_loss_state','hair_length_current_hair_length', 'hair_length_hair_goal', 'country', 'hair_type',
                  'how_often_do_you_heatstyling_tools', 'how_often_do_you_tight_hairstyle', 'how_often_do_you_hair_moisturizer',
       'how_often_do_you_scalp_massages', 'how_often_do_you_hair_wash',
       'occurrence_of_hair_breakage', 'causes_of_hair_breakage', 'occurrence_of_hair_breakage']

norminal_columns = [ 'race', 'gender', 'hair_edges_condition','hair_look', 'scalp_condition',
                    'is_your_hair_chemically_treated',
       'professional_treatments', 'protective_hairstyles_no_1','protective_hairstyles_no_2', 'condition_of_protective_hairstyles_used',
       'protective_hairstyles_maintenance','causes_of_hair_breakage',  'comb_type','detangling_style', 'eating_diet',
       'hair_state_and_their_cause_hydrated__healthy',
       'hair_state_and_their_cause_promote_frizzy',
       'hair_state_and_their_cause_tangled',
       'hair_state_and_their_cause_dryness__breaking']

sentence_columns = [ 'ingredient_promotes_your_hair_health','other_please_specify','hair_supplement_used',
                      'medication_or_condition_affecting_hair_growth','hair_or_scalp_allergies',
                       'tips_or_products_have_worked_well_for_your_hair', 'main_factor_influencing_your_hair_health_or_growth']
Binary_columns = ['keratin_treatment', 'family_history_of_hair_loss_or_slow_growth', 'satin_scarfbonnet_or_pillowcase', ]


## Data encoding

In [41]:
df = Survey_df
# -----------------------------------------------------
# 1. Numerical Column(s)
# -----------------------------------------------------
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# -----------------------------------------------------
# 2. Ordinal Columns → Gray Encoding
# -----------------------------------------------------
def to_gray_code(n):
    return n ^ (n >> 1)

def ordinal_to_gray(series):
    unique_vals = sorted(series.dropna().unique())
    mapping = {val: i for i, val in enumerate(unique_vals)}
    return series.map(mapping).fillna(0).astype(int).apply(to_gray_code)

for col in ordinal_columns:
    df[col] = ordinal_to_gray(df[col].astype(str).str.lower().str.strip())

# -----------------------------------------------------
# 3. Nominal Columns → Binary Encoding
# -----------------------------------------------------
def binary_encode_column(df, column):
    df[column] = df[column].astype(str).str.lower().str.strip()
    lb = LabelBinarizer()
    binary_df = pd.DataFrame(lb.fit_transform(df[column]), columns=[f"{column}_{cls}" for cls in lb.classes_])
    return binary_df

binary_encoded_nominals = [binary_encode_column(df, col) for col in norminal_columns]
df_nominal_binary = pd.concat(binary_encoded_nominals, axis=1)

In [47]:
# -----------------------------------------------------
# 4. Multi-word Sentence Columns → USE Embedding
# -----------------------------------------------------
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2025-07-23 22:52:36.504465: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-07-23 22:52:36.504545: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-07-23 22:52:36.504558: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-07-23 22:52:36.504762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-23 22:52:36.505321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-07-23 22:52:38.690883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


# Getting data from the database

In [51]:
df_final.to_csv("haircare_encoded_final2.csv", index=False)

In [9]:
import mysql.connector
import pandas as pd

# Connect
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="shalongo",
    database="AI_HAIR_ASSIST"
)

# Load full table into DataFrame
df = pd.read_sql("SELECT * FROM hairsurvey", conn)

# Close connection
conn.close()

# Display
#df.head()

/var/folders/sd/2tmrf4ds7nqg7lk_wdy856vm0000gn/T/ipykernel_27444/461702253.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM hairsurvey", conn)


## Feature Engineering with constant columns

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelBinarizer
import joblib


class SurveyEncoder:
    def __init__(self, numeric_columns, ordinal_columns, nominal_columns, binary_columns):
        self.numeric_columns = numeric_columns
        self.ordinal_columns = ordinal_columns
        self.nominal_columns = nominal_columns
        self.binary_columns = binary_columns

        self.scaler = StandardScaler()
        self.ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        self.nominal_encoders = {}
        self.feature_columns_ = None
        self.fitted = False

    # --- helper for label binarizer ---
    def _fit_label_binarizer(self, series, col):
        """Fit LabelBinarizer and store encoder"""
        series = series.astype(str).str.lower().str.strip()
        lb = LabelBinarizer()
        lb.fit(series)
        self.nominal_encoders[col] = lb

    def _transform_label_binarizer(self, series, col):
        """Transform series using already fitted LabelBinarizer"""
        lb = self.nominal_encoders[col]
        series = series.astype(str).str.lower().str.strip()

        # unseen categories → mark as zero vector
        known_classes = set(lb.classes_)
        mask_unknown = ~series.isin(known_classes)
        series[mask_unknown] = lb.classes_[0]  # fallback to first known class

        arr = lb.transform(series)
        if arr.ndim == 1:
            arr = arr.reshape(-1, 1)

        # consistent column names
        if len(lb.classes_) == 2:
            col_names = [f"{col}_{lb.classes_[1]}"]
        else:
            col_names = [f"{col}_{cls}" for cls in lb.classes_]

        df_enc = pd.DataFrame(arr, columns=col_names, index=series.index)
        return df_enc

    # --- fitting ---
    def fit(self, df):
        # Fit numeric scaler
        self.scaler.fit(df[self.numeric_columns])

        # Fit ordinal encoder
        self.ordinal_encoder.fit(df[self.ordinal_columns])

        # Fit label binarizers
        for col in self.nominal_columns + self.binary_columns:
            self._fit_label_binarizer(df[col], col)

        # Create feature column order for reference
        X_encoded = self._transform_internal(df)
        self.feature_columns_ = X_encoded.columns.tolist()

        self.fitted = True
        return self

    # --- internal transform (no column reindex) ---
    def _transform_internal(self, df):
        parts = []

        # Numeric
        num_scaled = self.scaler.transform(df[self.numeric_columns])
        parts.append(pd.DataFrame(num_scaled, columns=self.numeric_columns, index=df.index))

        # Ordinal
        ord_enc = self.ordinal_encoder.transform(df[self.ordinal_columns])
        parts.append(pd.DataFrame(ord_enc, columns=self.ordinal_columns, index=df.index))

        # Nominal + Binary
        for col in self.nominal_columns + self.binary_columns:
            df_enc = self._transform_label_binarizer(df[col], col)
            parts.append(df_enc)

        encoded_df = pd.concat(parts, axis=1)
        return encoded_df

    # --- public transform ---
    def transform(self, df):
        if not self.fitted:
            raise ValueError("Encoder must be fitted before transform.")
        encoded_df = self._transform_internal(df)

        # Ensure same columns and order as training
        encoded_df = encoded_df.reindex(columns=self.feature_columns_, fill_value=0)
        return encoded_df

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

    # --- save/load helpers ---
    def save(self, path):
        """Save encoder to disk."""
        joblib.dump(self, path)

    @staticmethod
    def load(path):
        """Load encoder from disk."""
        return joblib.load(path)

# Define columns to encode,

numeric_columns = [ 'consumed_waterday_l']

ordinal_columns = [ 'current_hair_condition', 'age', 'hair_porosity','hair_texture', 'hair_density', 'harline_condition',
                 'hair_breakage', 'hair_loss_state','hair_length_current_hair_length', 'hair_length_hair_goal', 'country', 'hair_type',
                  'how_often_do_you_heatstyling_tools', 'how_often_do_you_tight_hairstyle', 'how_often_do_you_hair_moisturizer',
       'how_often_do_you_scalp_massages', 'how_often_do_you_hair_wash',
       'occurrence_of_hair_breakage', 'causes_of_hair_breakage', 'occurrence_of_hair_breakage']

nominal_columns = [ 'race', 'gender', 'hair_edges_condition','hair_look', 'scalp_condition',
                    'is_your_hair_chemically_treated',
       'professional_treatments', 'protective_hairstyles_no_1','protective_hairstyles_no_2', 'condition_of_protective_hairstyles_used',
       'protective_hairstyles_maintenance','causes_of_hair_breakage',  'comb_type','detangling_style', 'eating_diet',
       'hair_state_and_their_cause_hydrated__healthy',
       'hair_state_and_their_cause_promote_frizzy',
       'hair_state_and_their_cause_tangled',
       'hair_state_and_their_cause_dryness__breaking']

sentence_columns = [ 'ingredient_promotes_your_hair_health','other_please_specify','hair_supplement_used',
                      'medication_or_condition_affecting_hair_growth','hair_or_scalp_allergies',
                       'tips_or_products_have_worked_well_for_your_hair', 'main_factor_influencing_your_hair_health_or_growth']
binary_columns = ['keratin_treatment', 'family_history_of_hair_loss_or_slow_growth', 'satin_scarfbonnet_or_pillowcase', ]

# initialize
encoder = SurveyEncoder(numeric_columns, ordinal_columns, nominal_columns, binary_columns)

# fit on training data
encoded_train = encoder.fit_transform(train_df)

# save encoder
encoder.save("survey_encoder.pkl")

# later...
encoder = SurveyEncoder.load("survey_encoder.pkl")

# transform a single new row (still produces same 171 columns)
encoded_row = encoder.transform(new_row_df)
